In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")


using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using GLMakie
using LinearAlgebra, LinearSolve,Statistics

cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab")
I0_hom = load("I0_hom_hom.jld")["data"]
u0_hom = load("U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

In [2]:
function makeevent(fp, ϵbox)
    function condition(u, t, integrator)
        x = Vector(u)
        b = x - fp[1]
        b = Vector(b)
        linprob = LinearProblem(A, b)
        linsolve = solve(linprob)
        return norm(linsolve.u, Inf) - ϵ_box
    end
    return condition
end
affect!(integrator) = terminate!(integrator)
function get_norm_αs(u)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf), linsolve
end
function color_alpha(alpha)
    if alpha > 0.0
        return "red"
    else
        return "blue"
    end
end;

In [3]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305

t = 3000.0; tspan = [0.0, t];

In [4]:
# index point from homoclinic curve
index_hom = 1
I0 = I0_hom[index_hom]; U0 = u0_hom[index_hom]

p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [0.0, 0.0, 0.0]

ds = CoupledODEs(TM, u0, p)
fp, _, _ = fixedpoints(ds, box, jacob_TM_)

ϵ_box = 1e-5
Ju0 = jacob_TM_(fp[1], p, 0);
fp_ = fp[1]

eigen_val_vec = eigen(Ju0)
eigen_vectors = eigen_val_vec.vectors

v1 = real(eigen_vectors[:, 1])
v2 = real(eigen_vectors[:, 2])
v3 = imag(eigen_vectors[:, 3])

v1 = normalize(v1, 2)
v1t = transpose(v1)

v2 = normalize(v2, 2)
v2t = transpose(v2)

v3 = normalize(v3, 2)
v3t = transpose(v3)

A = transpose([v1t; v2t; v3t])
A = Matrix(A)

3×3 Matrix{Float64}:
  0.998728    0.999899     0.0
 -0.0489879  -0.0141814    0.984697
  0.0119121   0.000705852  0.174274

In [5]:
total_number_points = 50
number_points_on_side = Int((total_number_points) / 1)

dots_right_side = zeros(number_points_on_side, length(u0))
array_α_vec_right = zeros(number_points_on_side, length(u0))

dots_left_side = zeros(number_points_on_side, length(u0))
array_α_vec_left = zeros(number_points_on_side, length(u0))

dots_up_side = zeros(number_points_on_side, length(u0))
array_α_vec_up = zeros(number_points_on_side, length(u0))

dots_down_side = zeros(number_points_on_side, length(u0))
array_α_vec_down = zeros(number_points_on_side, length(u0));

In [6]:
# case α1 = 0; α2 = ϵ_box; α3 = [-ϵ_box; ϵ_box]
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = ϵ_box
    α3 = - ϵ_box + 2 * ϵ_box * n / N
    α_vec = [α1, α2, α3]
    point = fp_ + A * α_vec
    
    dots_right_side[n, :] = point
    _, α_vec = get_norm_αs( dots_right_side[n, :] )
    array_α_vec_right[n, :] = α_vec
    n+=1
end

# case α1 = 0; α2 = -ϵ_box; α3 = [-ϵ_box; ϵ_box]
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = -ϵ_box
    α3 = - ϵ_box + 2 * ϵ_box * n / N
    α_vec = [α1, α2, α3]
    point = fp_ + A * α_vec
    
    dots_left_side[n, :] = point
    _, α_vec = get_norm_αs( dots_left_side[n, :] )
    array_α_vec_left[n, :] = α_vec
    n+=1
end

# case α1 = 0; α2 = [-ϵ_box; ϵ_box]; α3 = ϵ_box
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = - ϵ_box + 2 * ϵ_box * n / N
    α3 = ϵ_box
    α_vec = [α1, α2, α3]
    point = fp_ + A * α_vec
    
    dots_up_side[n, :] = point
    _, α_vec = get_norm_αs( dots_up_side[n, :] )
    array_α_vec_up[n, :] = α_vec
    n+=1
end

# case α1 = 0; α2 = [-ϵ_box; ϵ_box]; α3 = -ϵ_box
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = - ϵ_box + 2 * ϵ_box * n / N
    α3 = -ϵ_box
    α_vec = [α1, α2, α3]
    point = fp_ + A * α_vec
    
    dots_down_side[n, :] = point
    _, α_vec = get_norm_αs( dots_down_side[n, :] )
    array_α_vec_down[n, :] = α_vec
    n+=1
end;

In [7]:
event = makeevent(fp, ϵ_box)

(::var"#condition#20"{StateSpaceSet{3, Float64}}) (generic function with 1 method)

In [8]:
index = 1
cb = ContinuousCallback(event, nothing, affect!)
u0 = SA[dots_up_side[index, 1], dots_up_side[index, 2], dots_up_side[index, 3]]

prob = ODEProblem(TM, u0, tspan, p)
sol = solve(prob, alg = Vern9(), abstol = 1e-14, reltol = 1e-14, callback = cb)

retcode: Terminated
Interpolation: specialized 9th order lazy interpolation
t: 280433-element Vector{Float64}:
    0.0
    0.030805382554545225
    0.0493471258944327
    0.06907923400876988
    0.09159462786475568
    0.1110208686324919
    0.13162323181807667
    0.15437041828689635
    0.17509139256626885
    0.19657412951512795
    ⋮
 2891.307161128974
 2891.3271450900934
 2891.3472594048626
 2891.3718816678447
 2891.392654175351
 2891.4143344781564
 2891.43610911502
 2891.4539810580054
 2891.4539810580054
u: 280433-element Vector{SVector{3, Float64}}:
 [8.345798852588567, 0.7385046477659781, 0.43830029631861533]
 [8.346264207852645, 0.7384978316878643, 0.4383005865809681]
 [8.346539301191275, 0.7384928811357127, 0.43830059511825914]
 [8.346801225227058, 0.7384873018440303, 0.4383004500442738]
 [8.347032021764644, 0.7384810600706856, 0.4383000876612758]
 [8.347150025081955, 0.7384762609833265, 0.43829961790181227]
 [8.347174575017267, 0.7384722515887228, 0.4382989874020922]
 [8.347

In [9]:
norm_, alpha_ = get_norm_αs(sol[end])

(1.0000000007083036e-5, [3.057867884496885e-6, 1.0000000007083036e-5, -9.718679697691452e-6])

In [45]:
width, height = 600, 500
markersize = 5.0
color = :black
xlabel = L"x"; ylabel = L"y"; zlabel = L"E"
xlabelsize = 25; ylabelsize = 25; zlabelsize = 25

tstart, tend = length(sol)-500, length(sol)
f = Figure(resolution = (width, height))
#ax = Axis3(f[1, 1], xlabel = xlabel, ylabel = ylabel, zlabel = zlabel, xlabelsize = xlabelsize, ylabelsize = ylabelsize, zlabelsize = zlabelsize)

ax = LScene(f[1, 1], show_axis = true)
scale!(ax.scene, 50, 50, 1)

lines!(sol[2, tstart:tend], sol[3, tstart:tend], sol[1, tstart:tend], color = :deeppink, linewidth = 1.0)


scatter!(ax, dots_right_side[:, 2], dots_right_side[:, 3], dots_right_side[:, 1],  markersize = markersize, color = color)
scatter!(ax, dots_left_side[:, 2], dots_left_side[:, 3], dots_left_side[:, 1],  markersize = markersize, color = color)
scatter!(ax, dots_up_side[:, 2], dots_up_side[:, 3], dots_up_side[:, 1],  markersize = markersize, color = color)
scatter!(ax, dots_down_side[:, 2], dots_down_side[:, 3], dots_down_side[:, 1],  markersize = markersize, color = color)

scatter!(ax, fp_[2], fp_[3], fp_[1],  markersize = markersize, color = :blue)
display(GLMakie.Screen(), f)

GLMakie.Screen(...)

In [43]:
width, height = 600, 500
markersize = 5.0
color = color_alpha(alpha_[1])
xlabel = L"α_2"; ylabel = L"α_3"
xlabelsize = 25; ylabelsize = 25

f = Figure(resolution = (width, height))
ax = Axis(f[1, 1], xlabel = xlabel, ylabel = ylabel, xlabelsize = xlabelsize, ylabelsize = ylabelsize)

scatter!(ax, alpha_[2], alpha_[3], markersize = markersize, color = color)


scatter!(ax, 0.0, 0.0, markersize = markersize, color = :green)
display(GLMakie.Screen(), f)

GLMakie.Screen(...)

In [11]:
width, height = 600, 500
markersize = 5.0
color = :black
xlabel = L"x"; ylabel = L"y"; zlabel = L"E"
xlabelsize = 25; ylabelsize = 25; zlabelsize = 25

f = Figure(resolution = (width, height))
ax = Axis3(f[1, 1], xlabel = xlabel, ylabel = ylabel, zlabel = zlabel, xlabelsize = xlabelsize, ylabelsize = ylabelsize, zlabelsize = zlabelsize)
scatter!(ax, dots_right_side[:, 2], dots_right_side[:, 3], dots_right_side[:, 1],  markersize = markersize, color = color)
scatter!(ax, dots_left_side[:, 2], dots_left_side[:, 3], dots_left_side[:, 1],  markersize = markersize, color = color)
scatter!(ax, dots_up_side[:, 2], dots_up_side[:, 3], dots_up_side[:, 1],  markersize = markersize, color = color)
scatter!(ax, dots_down_side[:, 2], dots_down_side[:, 3], dots_down_side[:, 1],  markersize = markersize, color = color)

scatter!(ax, fp_[2], fp_[3], fp_[1],  markersize = markersize, color = :blue)
display(GLMakie.Screen(), f)

GLMakie.Screen(...)